# Optimizing your CRUD Operations

In [ ]:
# use the m201 database
use m201

In [ ]:
# create an explainable object
var exp = db.restaurants.explain("executionStats")

In [ ]:
# run an explained query (COLLSCAN & in-memory sort)
exp.find({ "address.zipcode": { $gt: '50000' }, cuisine: 'Sushi' }).sort({ stars: -1 })

In [ ]:
# create a naive index
db.restaurants.createIndex({"address.zipcode": 1,"cuisine": 1,"stars": 1})

In [ ]:
# rerun the query (uses the index, but isn't very selective and still does an 
# in-memory sort)
exp.find({ "address.zipcode": { $gt: '50000' }, cuisine: 'Sushi' }).sort({ stars: -1 })

In [ ]:
# see how many documents match 50000 for zipcode (10)
db.restaurants.find({ "address.zipcode": '50000' }).count()

In [ ]:
# see how many documents match our range (about half)
db.restaurants.find({ "address.zipcode": { $gt: '50000' } }).count()

In [ ]:
# see how many documents match an equality condition on cuisine (~2%)
db.restaurants.find({ cuisine: 'Sushi' }).count()

In [ ]:
# reorder the index key pattern to be more selective
db.restaurants.createIndex({ "cuisine": 1, "address.zipcode": 1, "stars": 1 })

In [ ]:
# and rerun the query (faster, still doing an in-memory sort)
exp.find({ "address.zipcode": { $gt: '50000' }, cuisine: 'Sushi' }).sort({ stars: -1 })

In [ ]:
# swap stars and zipcode to prevent an in-memory sort
db.restaurants.createIndex({ "cuisine": 1, "stars": 1, "address.zipcode": 1 })

In [ ]:
# awesome, no more in-memory sort! (uses the equality, sort, range rule)
exp.find({ "address.zipcode": { $gt: '50000' }, cuisine: 'Sushi' }).sort({ stars: -1 })

<img src="img/58.png">

<img src="img/59.png">

# Covered Queries

<img src="img/60.png">

In [ ]:
# use the m201 database
use m201

In [ ]:
# create an explainable object
var exp = db.restaurants.explain("executionStats")

In [ ]:
# create a compound index on three fields
db.restaurants.createIndex({name: 1, cuisine: 1, stars: 1})

In [ ]:
# checkout a projected query
db.restaurants.find({name: { $gt: 'L' }, cuisine: 'Sushi', stars: { $gte: 4.0 } }, { _id: 0, name: 1, cuisine: 1, stars: 1 })

In [ ]:
# and look at it's explain output (it's covered, no docs examined)
exp.find({name: { $gt: 'L' }, cuisine: 'Sushi', stars: { $gte: 4.0 } }, { _id: 0, name: 1, cuisine: 1, stars: 1 })

In [ ]:
# get the same output as the first query
# 注意這個 projection 跟之前的差別
db.restaurants.find({name: { $gt: 'L' }, cuisine: 'Sushi', stars: { $gte: 4.0 } }, { _id: 0, address: 0 })

In [ ]:
# but when looking at the explain output we see that it's not a covered query
exp.find({name: { $gt: 'L' }, cuisine: 'Sushi', stars: { $gte: 4.0 } }, { _id: 0, address: 0 })

---

### 範例01

In [ ]:
{ _id: 1 }
{ name: 1, dob: 1 }
{ hair: 1, name: 1 }

# Regex Performance

<img src="img/61.png">

<img src="img/62.png">

### 範例01

In [ ]:
db.products.createIndex({ productName: 1 })

In [ ]:
db.products.find({ productName: /^Craftsman/ })

# Aggregation Performance

<img src="img/63.png">

<img src="img/64.png">

### 範例01

### 範例02

In [ ]:
db.accounts.find( { accountBalance : { $gte : NumberDecimal(100000.00) }, city: "New York" } )
           .sort( { lastName: 1, firstName: 1 } )

In [ ]:
{ city: 1, lastName: 1, firstName: 1, accountBalance: 1 }